In [1]:
from pyspark import SparkConf ,SparkContext
from pyspark.sql.session import SparkSession


conf = SparkConf().setMaster("local").setAppName("word-counts")

sc = SparkContext(conf=conf)

In [2]:
import os
import re
d = {}
documentsProcessedRDD = sc.emptyRDD()
for i in range(len(os.listdir("dataset"))):
    bookRawRDD = sc.textFile("dataset/" + str(i))
    bookMessyTokensRDD = bookRawRDD.flatMap(lambda line: line.lower().strip().split(" "))
    bookCleanTokensRDD = bookMessyTokensRDD.map(lambda letter: re.sub(r'[^A-Za-z]', '', letter))
    
    #remove stop words
    from pyspark.ml.feature import StopWordsRemover
    remover = StopWordsRemover()
    stopwords = remover.getStopWords()
    bookwordsRDD = bookCleanTokensRDD.filter(lambda PointLessW: PointLessW not in stopwords)
    ##Remove the empty places
    bookEmptyRemoveRDD = bookwordsRDD.filter(lambda x: x != "")
    
    ### Appending the Individual book RDD to documentsProcessedRDD
    documentsProcessedRDD = documentsProcessedRDD.union(bookEmptyRemoveRDD)
    #print(documentsProcessedRDD.count())

In [4]:
print((documentsProcessedRDD.count()))

2741322


In [3]:
# map words to immediate keyvlue pairs
documentsProcessedPairsRDD = documentsProcessedRDD.map(lambda word: (word,1))

In [4]:
# tranform pairs to word count
documentsProcessedWordCountRDD = documentsProcessedPairsRDD.reduceByKey(lambda acc, value: acc + value)

In [16]:
# sort words in decending order
documentsProcessedResults = documentsProcessedWordCountRDD.map(lambda x: (x[1], x[0])).collect()
#print(documentsProcessedResults)

import pandas as pd
df = pd.DataFrame(documentsProcessedResults)
#df = df.insert(0, "ID", range(0, len(df)))
#print(df)
df = df.assign(New_ID=[ i for i in range(len(df))])[['New_ID'] + df.columns.tolist()]
del df[0]
df

New_ID               1
0            0           plays
1            1         william
2            2        official
3            3            byte
4            4             two
5            5          rather
6            6            away
7            7        directly
8            8           login
9            9       receiving
10          10           right
11          11      references
12          12          annual
13          13             ocr
14          14            high
15          15        pericles
16          16      everything
17          17             run
18          18            haue
19          19          vvould
20          20         procure
21          21      perfection
22          22        arraigne
23          23           envie
24          24          athens
25          25            envy
26          26             tis
27          27           muses
28          28          family
29          29            heat
...        ...             ...
117077  117077        kusamoto
117078  117078         longish
117079  117079        japannot
117080  117080        labentes
117081  117081        allround
117082  117082     arisenthere
117083  117083        paintbox
117084  117084     againindeed
117085  117085         sapiens
117086  117086          emmaus
117087  117087    trustfulness
117088  117088          romped
117089  117089    noblehearted
117090  117090   juniorsrather
117091  117091       ritualism
117092  117092         pieties
117093  117093  impenetrableat
117094  117094       compacter
117095  117095         pryersa
117096  117096        proctors
117097  117097     battersbywe
117098  117098       parentsoh
117099  117099   breakfastsand
117100  117100      chosewhich
117101  117101        myselfof
117102  117102         thingby
117103  117103      tendrement
117104  117104        startshe
117105  117105       sisterwas
117106  117106           grigg

[117107 rows x 2 columns]